<a href="https://colab.research.google.com/github/leejunho12316/HonGongMachine/blob/main/%ED%98%BC%EA%B3%B5%EB%A8%B8%EC%8B%A0_4_1_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#데이터 불러오기

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
fish = pd.read_csv('https://bit.ly/fish_csv_data')
fish

#데이터 탐색

In [ ]:
#물고기 종류
print(pd.unique(fish['Species']))

array(['Bream', 'Roach', 'Whitefish', 'Parkki', 'Perch', 'Pike', 'Smelt'],
      dtype=object)

In [ ]:
fish_input = fish[['Weight','Length','Diagonal','Height','Width']]
fish_target = fish['Species'].to_numpy()

In [ ]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(fish_input, fish_target, random_state = 42)

#KNN

In [ ]:
#정규화
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
ss.fit(train_input)

train_scaled = ss.transform(train_input)
test_scaled = ss.transform(test_input)

In [ ]:
#예측
from sklearn.neighbors import KNeighborsClassifier
kn = KNeighborsClassifier(n_neighbors = 3)
kn.fit(train_scaled, train_target)

print(kn.score(train_scaled, train_target))
print(kn.score(test_scaled, test_target))

0.8907563025210085
0.85


In [ ]:
print("클래스 순서 : \n", kn.classes_)
print(np.round(kn.predict_proba(test_scaled[:5]), decimals=4))

클래스 순서 : 
 ['Bream' 'Parkki' 'Perch' 'Pike' 'Roach' 'Smelt' 'Whitefish']
[[0.     0.     1.     0.     0.     0.     0.    ]
 [0.     0.     0.     0.     0.     1.     0.    ]
 [0.     0.     0.     1.     0.     0.     0.    ]
 [0.     0.     0.6667 0.     0.3333 0.     0.    ]
 [0.     0.     0.6667 0.     0.3333 0.     0.    ]]


In [ ]:
distances , indexes = kn.kneighbors(test_scaled[3:4])
print(distances, indexes)
print(train_target[indexes])

[[0.20774583 0.24862983 0.33682411]] [[104 115 106]]
[['Roach' 'Perch' 'Perch']]


#로지스틱 회귀 - 이진분류

In [ ]:
#bream, smelt 두 가지만 남겨
bream_smelt_indexes = (train_target == 'Bream') | (train_target == "Smelt")
train_bream_smelt = train_scaled[bream_smelt_indexes]
target_bream_smelt = train_target[bream_smelt_indexes]

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(train_bream_smelt, target_bream_smelt)

LogisticRegression()

In [ ]:
from scipy.special import expit
print("1. 로지스틱 회귀가 학습한 방정식 계수 : ", lr.coef_, lr.intercept_)
print("2. 방정식으로 구하는 z값 : ", lr.decision_function(train_bream_smelt[:5]))
print("3. z 값 Sigmoid 함수 적용 : ", lr.predict_proba(train_bream_smelt[:5]))
print("3-1. scipy로 Sigmoid 함수 적용 : ", expit(lr.decision_function(train_bream_smelt[:5])))

print("\n\n클래스 종류 : ", lr.classes_)
print("확률 계산 :", lr.predict(train_bream_smelt[:5]))

1. 로지스틱 회귀가 학습한 방정식 계수 :  [[-0.40451732 -0.57582787 -0.66248158 -1.01329614 -0.73123131]] [-2.16172774]
2. 방정식으로 구하는 z값 :  [-6.02991358  3.57043428 -5.26630496 -4.24382314 -6.06135688]
3. z 값 Sigmoid 함수 적용 :  [[0.99760007 0.00239993]
 [0.02737325 0.97262675]
 [0.99486386 0.00513614]
 [0.98585047 0.01414953]
 [0.99767419 0.00232581]]
3-1. scipy로 Sigmoid 함수 적용 :  [0.00239993 0.97262675 0.00513614 0.01414953 0.00232581]


클래스 종류 :  ['Bream' 'Smelt']
확률 계산 : ['Bream' 'Smelt' 'Bream' 'Bream' 'Bream']


#로지스틱 회귀 - 다중 분류

In [ ]:
lr = LogisticRegression(C=20, max_iter = 1000)
lr.fit(train_scaled, train_target)
print(lr.score(train_scaled, train_target))
print(lr.score(test_scaled, test_target))

0.9327731092436975
0.925


In [ ]:
from scipy.special import softmax
print("1. 로지스틱 회귀가 예측한 방정식 계수 (7개의 클래스 각각 방정식 하나씩 다 구함): ")
print(lr.coef_)
print(lr.intercept_)
print("2. 방정식으로 구하는 z값\n", np.round(lr.decision_function(test_scaled[:5]), decimals = 3) )
print("3. z값 Softmax 함수 적용\n", np.round(softmax(lr.decision_function(test_scaled[:5]), axis = 1)))

print("\n\n클래스 종류 : ", lr.classes_)
print("확률 계산 : \n", np.round(lr.predict_proba(test_scaled[:5]), decimals=3))
print("예측 결과 : ", lr.predict(test_scaled[:5]))

1. 로지스틱 회귀가 예측한 방정식 계수 (7개의 클래스 각각 방정식 하나씩 다 구함): 
[[-1.50605454 -1.03747913  2.60919714  7.69422736 -1.18603341]
 [ 0.19200048 -1.99988811 -3.79617726  6.50312639 -2.00022755]
 [ 3.55793539  6.36988929 -8.52233732 -5.75397234  3.79233439]
 [-0.11453307  3.61060121  3.94464503 -3.62243903 -1.75981681]
 [-1.40843715 -6.0924219   5.28629633 -0.86696568  1.84518453]
 [-1.33419959  1.48153394  1.38217548 -5.6602348  -4.39228962]
 [ 0.61328848 -2.33223529 -0.9037994   1.70625808  3.70084847]]
[-0.10345043 -0.27281216  3.24444852 -0.1756507   2.64960026 -6.72042857
  1.37829307]
2. 방정식으로 구하는 z값
 [[ -6.507   1.039   5.168  -2.757   3.337   0.346  -0.625]
 [-10.879   1.944   4.776  -2.423   2.987   7.842  -4.247]
 [ -4.338  -6.237   3.167   6.481   2.364   2.434  -3.872]
 [ -0.691   0.446   2.643  -1.215   3.261  -5.701   1.257]
 [ -6.401  -1.991   5.815  -0.135   3.499  -0.088  -0.699]]
3. z값 Softmax 함수 적용
 [[0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 